In [ ]:
pip install audiotsm

In [ ]:
# Imports
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import librosa
from librosa import display
import matplotlib.pyplot as plt
%matplotlib inline

import math
import pywt
from scipy import signal

from audiotsm import phasevocoder
from audiotsm.io.wav import WavReader, WavWriter

import json

In [ ]:
def read_audio_spectum_extended(filename):
    x, fs = librosa.load(filename)
    S = librosa.stft(x, N_FFT)
    S = np.log1p(np.abs(S)) 
    return S, x, fs

In [ ]:
# SOURCE: https://github.com/scaperot/the-BPM-detector-python/blob/master/bpm_detection/bpm_detection.py
def no_audio_data():
    print("No audio data for sample, skipping...")
    return None, None

def peak_detect(data):
    max_val = np.amax(abs(data))
    peak_ndx = np.where(data == max_val)
    if len(peak_ndx[0]) == 0:  # if nothing found then the max must be negative
        peak_ndx = np.where(data == -max_val)
    return peak_ndx

def bpm_detector(data, fs):
    cA = []
    cD = []
    correl = []
    cD_sum = []
    levels = 4
    max_decimation = 2 ** (levels - 1)
    min_ndx = math.floor(60.0 / 220 * (fs / max_decimation))
    max_ndx = math.floor(60.0 / 40 * (fs / max_decimation))

    for loop in range(0, levels):
        cD = []
        # 1) DWT
        if loop == 0:
            [cA, cD] = pywt.dwt(data, "db4")
            cD_minlen = len(cD) / max_decimation + 1
            cD_sum = np.zeros(math.floor(cD_minlen))
        else:
            [cA, cD] = pywt.dwt(cA, "db4")

        # 2) Filter
        cD = signal.lfilter([0.01], [1 - 0.99], cD)

        # 4) Subtract out the mean.

        # 5) Decimate for reconstruction later.
        cD = abs(cD[:: (2 ** (levels - loop - 1))])
        cD = cD - np.mean(cD)

        # 6) Recombine the signal before ACF
        #    Essentially, each level the detail coefs (i.e. the HPF values) are concatenated to the beginning of the array
        cD_sum = cD[0 : math.floor(cD_minlen)] + cD_sum

    if [b for b in cA if b != 0.0] == []:
        return no_audio_data()

    # Adding in the approximate data as well...
    cA = signal.lfilter([0.01], [1 - 0.99], cA)
    cA = abs(cA)
    cA = cA - np.mean(cA)
    cD_sum = cA[0 : math.floor(cD_minlen)] + cD_sum

    # ACF
    correl = np.correlate(cD_sum, cD_sum, "full")

    midpoint = math.floor(len(correl) / 2)
    correl_midpoint_tmp = correl[midpoint:]
    peak_ndx = peak_detect(correl_midpoint_tmp[min_ndx:max_ndx])
    if len(peak_ndx) > 1:
        return no_audio_data()

    peak_ndx_adjusted = peak_ndx[0] + min_ndx
    bpm = 60.0 / peak_ndx_adjusted * (fs / max_decimation)
    #print(bpm)
    return bpm, correl

def get_bpm(raw_samples, fs):
    data = []
    correl = []
    bpm = 0
    n = 0
    nsamps = len(raw_samples)
    window_samps = int(window * fs)
    samps_ndx = 0  # First sample in window_ndx
    max_window_ndx = math.floor(nsamps / window_samps)
    bpms = np.zeros(max_window_ndx)
    
    for window_ndx in range(0, max_window_ndx):

        # Get a new set of samples
        # print(n,":",len(bpms),":",max_window_ndx_int,":",fs,":",nsamps,":",samps_ndx)
        data = raw_samples[samps_ndx : samps_ndx + window_samps]
        if not ((len(data) % window_samps) == 0):
            raise AssertionError(str(len(data)))

        bpm, correl_temp = bpm_detector(data, fs)
        if bpm is None:
            continue
        bpms[window_ndx] = bpm
        correl = correl_temp

        # Iterate at the end of the loop
        samps_ndx = samps_ndx + window_samps

        # Counter for debug...
        n = n + 1

    bpm = np.median(bpms)
    
    return bpm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
CONTENT_FILENAME = '/content/drive/MyDrive/StyleChanger/content.wav'
STYLE_FILENAME = '/content/drive/MyDrive/StyleChanger/style.wav'
STYLE_BPM_MOD_FILENAME = '/content/drive/MyDrive/StyleChanger/style_synced.wav'

N_FFT = 2048

window = 3

In [ ]:
a_content, raw_content, fs_content = read_audio_spectum_extended(CONTENT_FILENAME)
a_style, raw_style, fs_style = read_audio_spectum_extended(STYLE_FILENAME)

In [ ]:
bpm_content = get_bpm(raw_content, fs_content);
bpm_style = get_bpm(raw_style, fs_style);
print(bpm_content)
print(bpm_style)

bpm_ratio = bpm_content / bpm_style;
print(bpm_ratio)

In [ ]:
with WavReader(STYLE_FILENAME) as reader:
    with WavWriter(STYLE_BPM_MOD_FILENAME, reader.channels, reader.samplerate) as writer:
        tsm = phasevocoder(reader.channels, speed=bpm_ratio)
        tsm.run(reader, writer)

In [ ]:
fp = open('/content/drive/MyDrive/StyleChanger/data.json', 'r')
settings = json.load(fp)
settings['styleBPM'] = bpm_style
settings['contentBPM'] = bpm_content
fp.close()

# save/overwrite file
fp = open('/content/drive/MyDrive/StyleChanger/data.json', 'w+')
fp.seek(0)
json.dump(settings, fp, indent=2)
fp.close()

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')